In [ ]:
from pickle_file import load_obj,save_obj
from tqdm import tqdm
import os
from datetime import datetime
import numpy as np
import networkx as nx
import igraph as ig
from ONet_fd import ONet_fd
import random as rd
import pandas as pd
import copy
#from prettytable import PrettyTable
## fd_set 文件位置：'obj_fd_set/fd_set_' + fd
## o_s_vector 文件位置： 'obj_osv_n/osv_n_' + fd, skill的顺序文件是 'obj_o_s_vector/skill_list'
fd_list = load_obj('folder_list')
#fd_increase1 = load_obj('fd_increase1')
fd_increase2 = load_obj('fd_increase2')
skill_list_std = load_obj('obj_osv_n/skill_list')
skill_dict = load_obj('skill_dict')
fd_time = load_obj('fd_time')


from influencial_spreader_node import unnormalized_osv_rca_weighted_ci,normalized_osv_rca_weighted_ci,jc_weighted_ci
from influencial_spreader_node import skillsim_weighted_ci
fd = fd_increase2[-1]

cognitive_score = load_obj('cognitive_score_6digit_osv_n/cognitive_score_6digit_osv_n_'+ fd)
#occupation_automation = load_obj('automation_occupation_frey_osborne')

save_time = '2020may'
city_codes = load_obj('occupations_city/' + save_time +'/city_codes_'+ save_time)
city_names = load_obj('occupations_city/' + save_time +'/city_names_'+ save_time)
city_occupations = load_obj('occupations_city/' + save_time +'/city_occupations_'+ save_time)
city_occupations_names = load_obj('occupations_city/' + save_time +'/city_occupations_names_'+ save_time)
city_emp = load_obj('occupations_city/' + save_time +'/city_emp_'+ save_time)
state_abbr = load_obj('occupations_city/' + save_time +'/state_abbr_'+ save_time)
city_county_names = load_obj('occupations_city/' + save_time +'/city_county_names_'+ save_time)
city_geo_dict = load_obj('occupations_city/' + save_time +'/msa_center_location')

city_occupations_wage_median = load_obj('occupations_city/' + save_time +'/city_occupations_wage_median_'+ save_time)
city_populations = load_obj('occupations_city/' + save_time +'/city_populations_'+ save_time)

occupations_6digit = load_obj('osv_6digit/occupations_6digit_'+fd)
occupations_6digit_names = load_obj('osv_6digit/occupations_6digit_names_'+fd)

# Figure6

In [ ]:
save_time = '2020may'
city_codes = load_obj('occupations_city/' + save_time +'/city_codes_'+ save_time)
city_names = load_obj('occupations_city/' + save_time +'/city_names_'+ save_time)
city_populations = load_obj('occupations_city/' + save_time +'/city_populations_'+ save_time)

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap,LogNorm

viridis = cm.get_cmap('bwr', 500)

import seaborn as sns

import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))

x = []
ss = 0
ts = 30
for i in range(ss,ts):
    x.append(i)

city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]
emp_no_PR = [city_populations[city] for city in city_codes_no_PR]
_, city_codes_no_PR_sort  = zip(*sorted(zip(emp_no_PR,city_codes_no_PR)))

import matplotlib as mpl
norm = mpl.colors.LogNorm(vmin=min(city_populations.values()), vmax=max(city_populations.values()))

font1 = {'family':'Times New Roman',
        'weight' : 'normal',
        'size'   : 23,
}
#legend = plt.legend(prop=font1)

y_5 = []
x_5 = []

for city in tqdm(city_codes_no_PR_sort):
    infection_number_empci_20 = load_obj('spreading_analysis/sp/spreading_analysis/spreading_simulation_empci_ratio__'+city+'__'+city+'_all')
    infection_number_random_20 = load_obj('spreading_analysis/sp/spreading_analysis/spreading_simulation_random_ratio__'+city+'__'+city+'_all')
    
    empci_list = []
    random_list = []
    for in_empci,in_random in zip(infection_number_empci_20,infection_number_random_20):
        temp_empci = []
        temp_random = []

        for empci_dl,random_dl in zip(in_empci,in_random):
            temp_empci.append(sum(empci_dl.values()))
            temp_random.append(sum(random_dl.values()))
            

        empci_list.append(temp_empci)
        random_list.append(temp_random)
        
    temp_relative = []
        
    for temp_empci, temp_random in zip(empci_list, random_list):
        temp = []
        for i in x:
            temp.append(temp_empci[i]/temp_random[i])
            
        temp_relative.append(temp)
        
        
    y = np.mean(np.array(temp_relative),0)
    
    
    plt.plot( x , y ,'-', lw=2,color = mpl.cm.ScalarMappable(norm=norm, cmap=viridis).to_rgba(city_populations[city])[0:3],alpha = 0.12)
    #plt.plot( x , y ,'r-', lw=2,alpha = (city_populations[city]+300000)/(300000+max(city_populations.values())))
    x_5.append(city_populations[city])
    y_5.append(y[4])


    
plt.plot([ss,ts],[1.0,1.0],'k--',lw=2)    
x_label_name = 'step'
y_label_name = 'relative effected number'

plt.xlabel(x_label_name,fontsize=31,weight='bold')
plt.ylabel(y_label_name,fontsize=31,weight='bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=28)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(4);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(4);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
#plt.xticks(np.arange(0,24,1))
#plt.yscale('log')
plt.ylim(0.97,1.15)
# 设置 X 轴的网格线，风格为 点画线
#plt.grid(axis='x',linestyle='-.')

font1 = {'family':'Times New Roman',
        'weight' : 'normal',
        'size'   : 23,
}
#legend = plt.legend(prop=font1)

#plt.savefig('empSI_all_cities.pdf', format = 'pdf')
plt.show()

# Figure7

see file "SIS_Figures"

# Figure8

In [ ]:
from scipy import stats

city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

city_list = []
population_temp = []
soc_cog_list = []
soc_empci_pearson = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    soc_cog_temp = []
    
    for i in range(0,int(len(ci_nodes_city_skillsim[city][1]))):
        o = ci_nodes_city_skillsim[city][1][i]
        ind_empci = ci_nodes_city_skillsim[city][1].index(o)
        empci = ci_nodes_city_skillsim[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            soc_cog_temp.append(cognitive_score[o])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    population_temp.append(city_populations[city])
    soc_cog_list.append(np.dot(empci_softmax_temp, np.array(soc_cog_temp)))
    soc_empci_pearson.append(np.corrcoef(soc_cog_temp, empci_temp)[0,1])
    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit
from scipy.stats import binned_statistic, pearsonr
from scipy.optimize import leastsq

import matplotlib as mpl

fig = plt.figure(figsize=(18,12))
ax = fig.add_subplot(111)
xt, yt = fig.transFigure.transform([1,1])

## x_0 = (x_temp[1:] + x_temp[:-1]) * .5


# def fit_func(p, x):
#     r,c,b = p
#     return 1/(np.log(x) - c)**r + b

# def relative_residuals(p,x,y):
#     return (fit_func(p,x)-y)/y


# p0 = [1,1,1] # 拟合的初始参数设置
# xR, yR = zip(*sorted(zip(population_temp,soc_cog_list)))

# para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
# yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

# print(para[0])

# plt.plot( xR, yF ,'k-', ms = 10, lw=2)


#plt.plot( population_temp, soc_empci_pearson,marker = 's', linestyle = '',color = 'red',ms = 10,alpha = 0.47)

print('correlation: ',pearsonr(population_temp, soc_empci_pearson))
#plt.plot( population_temp, soc_empci_spearman ,'rs', ms = 10, lw=2)


def fit_func(p, x):
    a,b = p
    return a*np.log(x) + b

def relative_residuals(p,x,y):
    return fit_func(p,x)-y


p0 = [1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,soc_empci_pearson)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

sns.lineplot( x = xR, y = yF , color = 'black', linestyle = '-',linewidth = 6)

x0 = [x for x in xR]
y0 = [0 for x in xR]
sns.lineplot( x = x0[0:2], y = y0[0:2] , color = 'black', ms = 2)


x_label_name = 'Population'
y_label_name = 'Correlation'
figure_name = 'Correlation between Cognitive Score and SI in Different Areas'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(6);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(6);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
plt.xlim(min(xR)-10000, max(xR))
plt.ylim(-0.1, 0.6)

xf_l = min(xR)-10000
xf_r = max(xR)
yf_l = min(yR) - 0.1
yf_r = max(yR) + 0.1

x_len = xf_r - xf_l
y_len = yf_r - yf_l

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = cm.get_cmap('magma', 500)

sb = 0.12
sa = 0.76

for x,y in zip(xR, yR):
    x2,y2 = ax.transData.transform((x,y))
    ax2 = fig.add_axes([x2/xt,y2/yt, 0.02, 0.02])
    #ax2.plot([0,1],[0,y], color=viridis(sb + sa * (y - min(yR))/(max(yR) - min(yR))),lw = 3)
    ax2.plot([0,1],[0,y], color='gray',lw = 3)
    ax2.spines['bottom'].set_linewidth(0)
    ax2.spines['left'].set_linewidth(0)
    ax2.spines['right'].set_linewidth(0)
    ax2.spines['top'].set_linewidth(0)
    ax2.set_xlim(0,1)
    ax2.set_ylim(min(yR), max(yR))
    ax2.axis('off')
    
    
#plt.savefig('CI_cog_cor_review_1.pdf',format = 'pdf')
plt.show()

# Figure9

## a

In [ ]:
city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

occupation_automation = load_obj('automation/automation_mit_6digit')

city_list = []
empci_auto_list = []
population_temp = []
soc_cog_list = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    soc_cog_temp = []
    
    for i in range(0,len(skillsim_emp_ci[city][1])):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o])
            soc_cog_temp.append(cognitive_score[o])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    empci_auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    population_temp.append(city_populations[city])
    soc_cog_list.append(np.dot(empci_softmax_temp, np.array(soc_cog_temp)))
    
    
import seaborn as sns

import matplotlib.pyplot as plt

plt.figure(figsize=(18,10))


#sns.set_theme(style='darkgrid')

#plt.plot( population_temp, empci_auto_list ,'rs', ms = 10, lw=2)

index_list = city_list
x_list = population_temp
y_list = empci_auto_list

data_list = population_temp

if (min(data_list)==0):
    data_log_min = 0
else:
    data_log_min = np.log10(min(data_list))

data_log_space = np.logspace(data_log_min, np.log10(max(data_list)), 13, base=10)

y_0, x_temp = np.histogram(data_list, data_log_space, density = True)

x_0 = (x_temp[1:] + x_temp[:-1]) * .5

x_order = np.digitize(population_temp, x_temp, right = True)

pop_bin = [x_0[i-1] for i in x_order]


x_a_s = pd.Series(dict(zip(index_list, population_temp)))
x_a = pd.Series(dict(zip(index_list, pop_bin)))
y_a = pd.Series(dict(zip(index_list, y_list)))

df = pd.DataFrame({'x_df':x_a, 'y_df':y_a, 'x_df_s':x_a_s})

# Plot the responses for different events and regions
palette = sns.color_palette("flare", 2)
sns.lineplot(x='x_df', y='y_df', data=df, color = 'red', linestyle='--', label = 'mean value',lw = 4)

sns.scatterplot(x='x_df_s', y='y_df', data=df, s = 120, alpha = 0.76)



from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
from scipy.optimize import leastsq

def fit_func(p, x):
    r,c,b = p
    return 1/(np.log(x) - c)**r + b

def relative_residuals(p,x,y):
    return (fit_func(p,x)-y)/y


p0 = [3,1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,empci_auto_list)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

#sns.lineplot( x = xR, y = yF , color = 'red', linestyle = '-', label = 'fitted curve')

x_label_name = 'population'
y_label_name = 'weighted automation'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(5);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(5);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
#plt.yscale('log')
plt.yticks(np.arange(0.59,0.61,0.01))
# 设置 X 轴的网格线，风格为 点画线
#plt.grid(axis='x',linestyle='-')

font1 = {'family':'Times New Roman',
        'weight' : 'bold',
        'size'   : 31,
}
legend = plt.legend(prop=font1, frameon = False)


#plt.savefig('weightauto_MIT.pdf', format = 'pdf')
plt.show()

In [ ]:
from scipy import stats

city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

city_list = []
population_temp = []
auto_list = []
auto_empci_pearson = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    
    for i in range(0,int(len(skillsim_emp_ci[city][1]))):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    population_temp.append(city_populations[city])
    auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    auto_empci_pearson.append(np.corrcoef(auto_temp, empci_temp)[0,1])
    
    

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit
from scipy.stats import binned_statistic, pearsonr
from scipy.optimize import leastsq

import matplotlib as mpl

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
xt, yt = fig.transFigure.transform([1,1])

#plt.plot( population_temp, auto_empci_pearson,marker = 's', linestyle = '',color = 'red',ms = 10,alpha = 0.47)

print('correlation: ',pearsonr(population_temp, auto_empci_pearson))
#plt.plot( population_temp, soc_empci_spearman ,'rs', ms = 10, lw=2)


def fit_func(p, x):
    a,b = p
    return a*np.log(x) + b

def relative_residuals(p,x,y):
    return fit_func(p,x)-y


p0 = [1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,auto_empci_pearson)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

sns.lineplot( x = xR, y = yF , color = 'black', linestyle = '-',linewidth = 6)

x0 = [x for x in xR]
y0 = [0 for x in xR]
sns.lineplot( x = x0[0:2], y = y0[0:2] , color = 'black', ms = 2)


x_label_name = 'Population'
y_label_name = 'Correlation'
figure_name = 'Correlation between Cognitive Score and SI in Different Areas'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(6);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(6);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
plt.xlim(min(xR)-10000, max(xR))
plt.ylim(-0.25, 0.1)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = cm.get_cmap('magma', 500)

sb = 0.12
sa = 0.76

for x,y in zip(xR, yR):
    x2,y2 = ax.transData.transform((x,y))
    ax2 = fig.add_axes([x2/xt,y2/yt, 0.02, 0.02])
    #ax2.plot([0,1],[0,y], color=viridis(sb + sa * (y - min(yR))/(max(yR) - min(yR))),lw = 3)
    ax2.plot([0,1],[0,y], color='gray',lw = 3)
    ax2.spines['bottom'].set_linewidth(0)
    ax2.spines['left'].set_linewidth(0)
    ax2.spines['right'].set_linewidth(0)
    ax2.spines['top'].set_linewidth(0)
    ax2.set_xlim(0,1)
    ax2.set_ylim(-0.25,0.1)
    ax2.axis('off')
    
    
#plt.savefig('empci_auto_correlation_sml_review_1.pdf',format = 'pdf',  bbox_inches='tight')
plt.show()

## b

In [ ]:
ci_nodes_city_skillsim = load_obj('skillsim_ci_results/ci_nodes_city_skillsim_0')

## 考虑emp
def emp_ci(ci_city):

    emp_ci_city = {}
    
    save_time = '2020may'
    city_codes = load_obj('occupations_city/' + save_time +'/city_codes_'+ save_time)
    city_names = load_obj('occupations_city/' + save_time +'/city_names_'+ save_time)
    city_occupations = load_obj('occupations_city/' + save_time +'/city_occupations_'+ save_time)
    city_occupations_names = load_obj('occupations_city/' + save_time +'/city_occupations_names_'+ save_time)
    city_emp = load_obj('occupations_city/' + save_time +'/city_emp_'+ save_time)

    for city in city_codes:
        temp_lbci = []
        temp_names = []

        for i in range(0, len(ci_city[city][1])):
            o = ci_city[city][1][i]
            ci = ci_city[city][0][i]
            temp_lbci.append(np.log(city_emp[city][o]) * ci)
            temp_names.append(o)


        weighted_ci_values, weighted_ci_names = zip(*sorted(zip(temp_lbci,temp_names), reverse = True))

        emp_ci_city[city] = [weighted_ci_values, weighted_ci_names]

    return emp_ci_city

skillsim_emp_ci = emp_ci(ci_nodes_city_skillsim)

In [ ]:
city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

occupation_automation = load_obj('automation/automation_webb_3_6digit')
items = 0

city_list = []
empci_auto_list = []
population_temp = []
soc_cog_list = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    soc_cog_temp = []
    
    for i in range(0,len(skillsim_emp_ci[city][1])):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o][items])
            soc_cog_temp.append(cognitive_score[o])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    empci_auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    population_temp.append(city_populations[city])
    soc_cog_list.append(np.dot(empci_softmax_temp, np.array(soc_cog_temp)))
    
import seaborn as sns

import matplotlib.pyplot as plt

plt.figure(figsize=(18,10))


#sns.set_theme(style='darkgrid')

#plt.plot( population_temp, empci_auto_list ,'rs', ms = 10, lw=2)

index_list = city_list
x_list = population_temp
y_list = empci_auto_list

data_list = population_temp

if (min(data_list)==0):
    data_log_min = 0
else:
    data_log_min = np.log10(min(data_list))

data_log_space = np.logspace(data_log_min, np.log10(max(data_list)), 13, base=10)

y_0, x_temp = np.histogram(data_list, data_log_space, density = True)

x_0 = (x_temp[1:] + x_temp[:-1]) * .5

x_order = np.digitize(population_temp, x_temp, right = True)

pop_bin = [x_0[i-1] for i in x_order]


x_a_s = pd.Series(dict(zip(index_list, population_temp)))
x_a = pd.Series(dict(zip(index_list, pop_bin)))
y_a = pd.Series(dict(zip(index_list, y_list)))

df = pd.DataFrame({'x_df':x_a, 'y_df':y_a, 'x_df_s':x_a_s})

# Plot the responses for different events and regions
palette = sns.color_palette("flare", 2)
sns.lineplot(x='x_df', y='y_df', data=df, color = 'red', linestyle='--', label = 'mean value',lw = 4)

sns.scatterplot(x='x_df_s', y='y_df', data=df, s = 120, alpha = 0.90, color = '#FDE482')



from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
from scipy.optimize import leastsq

def fit_func(p, x):
    r,c,b = p
    return 1/(np.log(x) - c)**r + b

def relative_residuals(p,x,y):
    return (fit_func(p,x)-y)/y


p0 = [3,1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,empci_auto_list)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

#sns.lineplot( x = xR, y = yF , color = 'red', linestyle = '-', label = 'fitted curve')

x_label_name = 'Population'
y_label_name = 'Weighted Automation'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(5);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(5);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
#plt.yscale('log')
#plt.xticks(np.arange(0,24,1))
# 设置 X 轴的网格线，风格为 点画线
#plt.grid(axis='x',linestyle='-')

font1 = {'family':'Times New Roman',
        'weight' : 'bold',
        'size'   : 31,
}
legend = plt.legend(prop=font1, frameon = False)


#plt.savefig('weightauto_webb_robot_review_1.pdf', format = 'pdf')
plt.show()
    

In [ ]:
from scipy import stats

city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

city_list = []
population_temp = []
auto_list = []
auto_empci_pearson = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    
    for i in range(0,int(len(skillsim_emp_ci[city][1]))):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o][items])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    population_temp.append(city_populations[city])
    auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    auto_empci_pearson.append(np.corrcoef(auto_temp, empci_temp)[0,1])
    
    
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit
from scipy.stats import binned_statistic, pearsonr
from scipy.optimize import leastsq

import matplotlib as mpl

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
xt, yt = fig.transFigure.transform([1,1])

#plt.plot( population_temp, auto_empci_pearson,marker = 's', linestyle = '',color = 'red',ms = 10,alpha = 0.47)

print('correlation: ',pearsonr(population_temp, auto_empci_pearson))
#plt.plot( population_temp, soc_empci_spearman ,'rs', ms = 10, lw=2)


def fit_func(p, x):
    a,b = p
    return a*np.log(x) + b

def relative_residuals(p,x,y):
    return fit_func(p,x)-y


p0 = [1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,auto_empci_pearson)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

sns.lineplot( x = xR, y = yF , color = 'black', linestyle = '-',linewidth = 6)

x0 = [x for x in xR]
y0 = [0 for x in xR]
sns.lineplot( x = x0[0:2], y = y0[0:2] , color = 'black', ms = 2)


x_label_name = 'Population'
y_label_name = 'Correlation'
figure_name = 'Correlation between Cognitive Score and SI in Different Areas'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(6);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(6);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
plt.xlim(min(xR)-10000, max(xR))
plt.ylim(-0.4, 0.1)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = cm.get_cmap('magma', 500)

sb = 0.12
sa = 0.76

for x,y in zip(xR, yR):
    x2,y2 = ax.transData.transform((x,y))
    ax2 = fig.add_axes([x2/xt,y2/yt, 0.02, 0.02])
    #ax2.plot([0,1],[0,y], color=viridis(sb + sa * (y - min(yR))/(max(yR) - min(yR))),lw = 3)
    ax2.plot([0,1],[0,y], color='gray',lw = 3)
    ax2.spines['bottom'].set_linewidth(0)
    ax2.spines['left'].set_linewidth(0)
    ax2.spines['right'].set_linewidth(0)
    ax2.spines['top'].set_linewidth(0)
    ax2.set_xlim(0,1)
    ax2.set_ylim(min(yR), max(yR))
    ax2.axis('off')
    
    
#plt.savefig('empci_auto_correlation_webb_robot_review_1.pdf',format = 'pdf', bbox_inches='tight')
plt.show()
    

## c

In [ ]:
city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

occupation_automation = load_obj('automation/automation_webb_3_6digit')
items = 2

city_list = []
empci_auto_list = []
population_temp = []
soc_cog_list = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    soc_cog_temp = []
    
    for i in range(0,len(skillsim_emp_ci[city][1])):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o][items])
            soc_cog_temp.append(cognitive_score[o])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    empci_auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    population_temp.append(city_populations[city])
    soc_cog_list.append(np.dot(empci_softmax_temp, np.array(soc_cog_temp)))
    
import seaborn as sns

import matplotlib.pyplot as plt

plt.figure(figsize=(18,10))


#sns.set_theme(style='darkgrid')

#plt.plot( population_temp, empci_auto_list ,'rs', ms = 10, lw=2)

index_list = city_list
x_list = population_temp
y_list = empci_auto_list

data_list = population_temp

if (min(data_list)==0):
    data_log_min = 0
else:
    data_log_min = np.log10(min(data_list))

data_log_space = np.logspace(data_log_min, np.log10(max(data_list)), 13, base=10)

y_0, x_temp = np.histogram(data_list, data_log_space, density = True)

x_0 = (x_temp[1:] + x_temp[:-1]) * .5

x_order = np.digitize(population_temp, x_temp, right = True)

pop_bin = [x_0[i-1] for i in x_order]


x_a_s = pd.Series(dict(zip(index_list, population_temp)))
x_a = pd.Series(dict(zip(index_list, pop_bin)))
y_a = pd.Series(dict(zip(index_list, y_list)))

df = pd.DataFrame({'x_df':x_a, 'y_df':y_a, 'x_df_s':x_a_s})

# Plot the responses for different events and regions
palette = sns.color_palette("flare", 2)
sns.lineplot(x='x_df', y='y_df', data=df, color = 'red', linestyle='--', label = 'mean value',lw = 4)

sns.scatterplot(x='x_df_s', y='y_df', data=df, s = 120, alpha = 0.90, color = '#FDE482')



from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
from scipy.optimize import leastsq

def fit_func(p, x):
    r,c,b = p
    return 1/(np.log(x) - c)**r + b

def relative_residuals(p,x,y):
    return (fit_func(p,x)-y)/y


p0 = [3,1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,empci_auto_list)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

#sns.lineplot( x = xR, y = yF , color = 'red', linestyle = '-', label = 'fitted curve')

x_label_name = 'Population'
y_label_name = 'Weighted Automation'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(5);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(5);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
#plt.yscale('log')
#plt.xticks(np.arange(0,24,1))
# 设置 X 轴的网格线，风格为 点画线
#plt.grid(axis='x',linestyle='-')

font1 = {'family':'Times New Roman',
        'weight' : 'bold',
        'size'   : 31,
}
legend = plt.legend(prop=font1, frameon = False)


#plt.savefig('weightauto_webb_robot_review_1.pdf', format = 'pdf')
plt.show()
    

In [ ]:
from scipy import stats

city_codes_no_PR = [city for city in city_codes if city_names[city][-2:] != 'PR' and city in city_populations.keys()]

city_list = []
population_temp = []
auto_list = []
auto_empci_pearson = []


for city in city_codes_no_PR:
    empci_temp = []
    occu_temp = []
    auto_temp = []
    
    for i in range(0,int(len(skillsim_emp_ci[city][1]))):
        o = skillsim_emp_ci[city][1][i]
        ind_empci = skillsim_emp_ci[city][1].index(o)
        empci = skillsim_emp_ci[city][0][ind_empci]

        emp = city_emp[city][o]
        wage = city_occupations_wage_median[city][o]

        if (o in occupations_6digit) and (emp != 0) and (wage != 0) and (o in occupation_automation.keys()):
            empci_temp.append(empci)
            occu_temp.append(o)
            auto_temp.append(occupation_automation[o][items])
                
    empci_softmax_temp = np.array(empci_temp) / np.sum(np.array(empci_temp))
    
    city_list.append(city)
    population_temp.append(city_populations[city])
    auto_list.append(np.dot(empci_softmax_temp, np.array(auto_temp)))
    auto_empci_pearson.append(np.corrcoef(auto_temp, empci_temp)[0,1])
    
    
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit
from scipy.stats import binned_statistic, pearsonr
from scipy.optimize import leastsq

import matplotlib as mpl

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
xt, yt = fig.transFigure.transform([1,1])

#plt.plot( population_temp, auto_empci_pearson,marker = 's', linestyle = '',color = 'red',ms = 10,alpha = 0.47)

print('correlation: ',pearsonr(population_temp, auto_empci_pearson))
#plt.plot( population_temp, soc_empci_spearman ,'rs', ms = 10, lw=2)


def fit_func(p, x):
    a,b = p
    return a*np.log(x) + b

def relative_residuals(p,x,y):
    return fit_func(p,x)-y


p0 = [1,1] # 拟合的初始参数设置
xR, yR = zip(*sorted(zip(population_temp,auto_empci_pearson)))

para =leastsq(relative_residuals, p0, args=(xR,yR)) # 进行拟合
yF = np.fromiter( ( fit_func( para[0], xx ) for xx in xR ), float)

print(para[0])

sns.lineplot( x = xR, y = yF , color = 'black', linestyle = '-',linewidth = 6)

x0 = [x for x in xR]
y0 = [0 for x in xR]
sns.lineplot( x = x0[0:2], y = y0[0:2] , color = 'black', ms = 2)


x_label_name = 'Population'
y_label_name = 'Correlation'
figure_name = 'Correlation between Cognitive Score and SI in Different Areas'

plt.xlabel(x_label_name,fontsize=41,weight = 'bold')
plt.ylabel(y_label_name,fontsize=41,weight = 'bold')
#plt.title(figure_name,fontsize=30)
# ####设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=38)
#plt.legend()
ax=plt.gca();#获得坐标轴的句柄
ax.spines['bottom'].set_linewidth(6);###设置底部坐标轴的粗细
ax.spines['left'].set_linewidth(6);####设置左边坐标轴的粗细
ax.spines['right'].set_linewidth(0);###设置右边坐标轴的粗细
ax.spines['top'].set_linewidth(0);####设置上部坐标轴的粗细
plt.xscale('log')
plt.xlim(min(xR)-10000, max(xR))
plt.ylim(-0.4, 0.1)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

viridis = cm.get_cmap('magma', 500)

sb = 0.12
sa = 0.76

for x,y in zip(xR, yR):
    x2,y2 = ax.transData.transform((x,y))
    ax2 = fig.add_axes([x2/xt,y2/yt, 0.02, 0.02])
    #ax2.plot([0,1],[0,y], color=viridis(sb + sa * (y - min(yR))/(max(yR) - min(yR))),lw = 3)
    ax2.plot([0,1],[0,y], color='gray',lw = 3)
    ax2.spines['bottom'].set_linewidth(0)
    ax2.spines['left'].set_linewidth(0)
    ax2.spines['right'].set_linewidth(0)
    ax2.spines['top'].set_linewidth(0)
    ax2.set_xlim(0,1)
    ax2.set_ylim(min(yR), max(yR))
    ax2.axis('off')
    
    
#plt.savefig('empci_auto_correlation_webb_robot_review_1.pdf',format = 'pdf', bbox_inches='tight')
plt.show()
    